In [17]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
from youtube_transcript_api import YouTubeTranscriptApi
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import json
import re

In [2]:
with open('videos_galaxy2.json', 'r') as file:
    videos = json.load(file)

In [182]:
videos['0']

{'videoId': 'BSYsXVFzmKA',
 'title': 'Galaxy S23 Ultra: Official Introduction Film | Samsung',
 'description': "What's new? The new Galaxy S23 Ultra. Share the epic with our most powerful processor yet, a pro-grade camera that boasts epic ...",
 'viewCount': '24328894',
 'captions': "hi! Nothing bad, right? what's the matter? braided!  Galaxy S23 Ultra. Good. This cannot be expressed simply as ‘good.’ First, the\ndisplay color and contrast are automatically adjusted to match the content you are watching or the surrounding environment. It’s a function called ‘Vision Booster’. Oh really? Now, look! and…  ! Can you drive this? look!  Snapdragon 8 2nd generation for Galaxy! Let's increase GPU, CPU, and NPU. It has the most powerful processor in the Galaxy. Stop, stop, stop! and…  !  I can't stop! This processor is efficient and can keep going. for a moment…  Are we in the water now? fountain!  Concentrate!  The Galaxy S23 Ultra is a product that takes the earth into consideration.  It cons

In [183]:
docs = [v.get('title') + ' '  + v.get('description') + ' '  + v.get('captions') for v in videos.values()]

In [184]:
docs[100]

"Samsung Galaxy S23 Ultra - THIS IS AWESOME Samsung Galaxy S23 Ultra will see some significant upgrades in camera, performance, and battery life. But the Galaxy S23 Ultra ... all right what's up guys Sal here so we've been talking about how the Galaxy s23 Ultra will offer some pretty significant upgrades in camera performance and even in battery life these three alone make the s23 ultra a substantial upgrade over its predecessor but you see the phone is not just about the camera and performance there are other important elements to it as well that people often Overlook when making a purchasing decision and one of those is the speaker quality most of us have phones in our pockets and use them for almost everything such as watching movies playing games and listening to music while some people may opt to use wireless headphones or Bluetooth speakers to enhance the audio experience the majority of users prefer the convenience of using the phone's built-in speakers so the speaker quality is

In [185]:
def preprocess(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='n') for token in filtered_tokens]
    lemmatized_tokens += [lemmatizer.lemmatize(token, pos='a') for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)
    return lemmatized_tokens

In [186]:
pdoc = [preprocess(doc) for doc in docs]

In [24]:
pdoc0 = processed_data

In [191]:
processed_data = pdoc

In [198]:
'really' in processed_data[101]

False

In [196]:
my_sw = ['really', 'samsung', 'samsungs', 'galaxy']

In [193]:
new_data = []
for doc in processed_data:
    new_data.append([word for word in doc if word not in my_sw])

In [197]:
processed_data = new_data

In [337]:
dictionary = corpora.Dictionary(processed_data)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in processed_data]

num_topics = 20
chunksize = 2000
passes = 20
iterations = 500
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token
'''
model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)
'''
model = LdaModel(corpus, id2word=id2word, iterations=iterations, num_topics=num_topics)

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -0.9229.
[([(0.008376592, 'lens'),
   (0.007450741, 'think'),
   (0.007407625, 'iphone'),
   (0.0071220864, 'little'),
   (0.006394853, 'pro'),
   (0.0062396266, 'bit'),
   (0.005572624, 'im'),
   (0.0054731453, 'going'),
   (0.0048601977, 'part'),
   (0.0045391694, 'feel'),
   (0.0045206095, 'color'),
   (0.004496204, 'zoom'),
   (0.0043786718, 'x'),
   (0.0043172953, 'device'),
   (0.004244151, 'right'),
   (0.0038506493, 'best'),
   (0.0037016103, 'still'),
   (0.0036758448, 'photo'),
   (0.0035453923, 'thats'),
   (0.0035030614, 'way')],
  -0.5527836694719998),
 ([(0.013365424, 'iphone'),
   (0.005567425, 'im'),
   (0.0049940543, 'device'),
   (0.004779054, 'little'),
   (0.0047451784, 'photo'),
   (0.0043437, 'zoom'),
   (0.004293931, 'color'),
   (0.0040494897, 'apple'),
   (0.004042278, 'think'),
   (0.0040271035, 'k'),
   (0.003953417, 'pro'),
   (0.0038955244, 'going'),
   (0.0037839282, 'difference'),
   (0.0037088608, 'lot'),
   (0.0036073911, 'mode'

In [339]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'file_name.html')

In [340]:
probabilities = [model.get_document_topics(c, minimum_probability=0) for c in corpus]
probabilities

[[(0, 0.00018200993),
  (1, 0.00018200993),
  (2, 0.00018200993),
  (3, 0.00018200993),
  (4, 0.00018200993),
  (5, 0.70465875),
  (6, 0.00018200993),
  (7, 0.00018200993),
  (8, 0.00018200993),
  (9, 0.00018200993),
  (10, 0.00018200993),
  (11, 0.00018200993),
  (12, 0.00018200993),
  (13, 0.00018200993),
  (14, 0.00018200993),
  (15, 0.00018200993),
  (16, 0.2920651),
  (17, 0.00018200993),
  (18, 0.00018200993),
  (19, 0.00018200993)],
 [(0, 4.7200225e-05),
  (1, 4.7200225e-05),
  (2, 4.7200225e-05),
  (3, 4.7200225e-05),
  (4, 4.7200225e-05),
  (5, 4.7200225e-05),
  (6, 0.99910325),
  (7, 4.7200225e-05),
  (8, 4.7200225e-05),
  (9, 4.7200225e-05),
  (10, 4.7200225e-05),
  (11, 4.7200225e-05),
  (12, 4.7200225e-05),
  (13, 4.7200225e-05),
  (14, 4.7200225e-05),
  (15, 4.7200225e-05),
  (16, 4.7200225e-05),
  (17, 4.7200225e-05),
  (18, 4.7200225e-05),
  (19, 4.7200225e-05)],
 [(0, 2.371877e-05),
  (1, 2.371877e-05),
  (2, 2.371877e-05),
  (3, 2.371877e-05),
  (4, 2.371877e-05),
  (

In [202]:
probabilities[0]

[(0, 0.00018205216),
 (1, 0.00018205216),
 (2, 0.00018205216),
 (3, 0.00018205216),
 (4, 0.00018205216),
 (5, 0.00018205217),
 (6, 0.00018205216),
 (7, 0.00018205216),
 (8, 0.00018205216),
 (9, 0.00018205216),
 (10, 0.99654096),
 (11, 0.00018205216),
 (12, 0.00018205216),
 (13, 0.00018205216),
 (14, 0.00018205216),
 (15, 0.00018205216),
 (16, 0.00018205216),
 (17, 0.00018205216),
 (18, 0.00018205216),
 (19, 0.00018205216)]

In [341]:
prob_arr = []
for prob in probabilities:
    tmp = []
    for p in prob:
        tmp.append(p[1])
    prob_arr.append(tmp)

In [60]:
import numpy as np

In [342]:
prob_arr = np.array(prob_arr)

In [343]:
prob_arr.shape

(901, 20)

In [345]:
prob_arr

array([[0.000182, 0.000182, 0.000182, ..., 0.000182, 0.000182, 0.000182],
       [0.000047, 0.000047, 0.000047, ..., 0.000047, 0.000047, 0.000047],
       [0.000024, 0.000024, 0.000024, ..., 0.000024, 0.000024, 0.000024],
       ...,
       [0.000027, 0.000027, 0.000027, ..., 0.000027, 0.000027, 0.000027],
       [0.000024, 0.000024, 0.000024, ..., 0.000024, 0.000024, 0.000024],
       [0.00001 , 0.00001 , 0.00001 , ..., 0.00001 , 0.00001 , 0.00001 ]],
      dtype=float32)

In [137]:
viewcnts = [v.get('viewCount') for v in videos.values()]

In [138]:
viewcnts = [int(v) for v in viewcnts]

In [150]:
viewcnts

[24328894,
 4105700,
 5608037,
 846498,
 360439,
 833664,
 373151,
 185620,
 102170,
 303152,
 139424,
 105863,
 2905101,
 21876031,
 533256,
 3210975,
 770435,
 1568714,
 2393519,
 1387867,
 13740,
 2316272,
 2290913,
 349401,
 281310,
 39636,
 161271,
 106333,
 306237,
 30081,
 1579584,
 5700,
 4607241,
 84367,
 1504108,
 602896,
 123576,
 65199,
 256998,
 246186,
 31679,
 1859636,
 384962,
 518761,
 186762,
 5340,
 55008,
 3379421,
 4456226,
 60091,
 7627,
 78446,
 1280029,
 77341,
 59927,
 310024,
 380982,
 171569,
 73061,
 135101,
 88048,
 104950,
 94203,
 340190,
 127541,
 8506,
 121686,
 29100,
 31047,
 46230,
 313995,
 1337782,
 3029687,
 99402,
 9659,
 158141,
 2705596,
 1839011,
 1451,
 493480,
 270454,
 159722,
 137935,
 87581,
 13459,
 130529,
 21073,
 1221712,
 90089,
 257827,
 9591811,
 707224,
 3694,
 39337,
 149497,
 260029,
 15413,
 15431,
 187360,
 1019,
 115156,
 59920,
 2093873,
 91165,
 6297,
 538482,
 60947,
 97205,
 383255,
 167941,
 103294,
 186154,
 215298,
 23

In [82]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [71]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [346]:
X = prob_arr
y = viewcnts
reg = LinearRegression().fit(X, y)

In [347]:
X.shape

(901, 20)

In [348]:
reg.predict(X)

array([1395294.     ,  311276.44   ,  296162.25   ,  446773.22   ,
        527952.6    ,  261853.73   ,  311279.5    ,  649423.9    ,
        257594.98   ,  277694.06   ,   48156.47   ,  420168.47   ,
        274121.22   , 1765421.9    ,  122246.95   ,  311088.84   ,
        343574.88   ,  320017.12   ,  182510.64   ,  258365.4    ,
        178968.52   ,  222655.1    ,  346346.88   ,  152377.88   ,
         90157.83   ,  912203.75   ,  310317.72   ,  267520.97   ,
        311244.     ,  302179.84   ,  657484.94   ,   86452.25   ,
        673787.6    ,  351949.7    ,  310317.53   ,  428368.1    ,
        122451.016  ,   35883.97   ,  686988.     ,   27531.719  ,
       1168412.1    ,  215832.1    ,  295441.06   ,  311230.53   ,
        -84409.34   ,  264653.53   ,   36755.844  ,  311279.56   ,
        219543.64   ,  578550.25   , 1039617.9    ,  554524.75   ,
        428223.62   ,  115351.69   ,  233097.12   ,  311185.66   ,
        157764.62   ,  584867.4    ,   70100.484  ,   51552.42

In [349]:
reg.score(X, y)

0.05202367954032461

In [350]:
reg.coef_.max()

1554221.1

In [351]:
reg.coef_.min()

-389396.56

In [261]:
np.set_printoptions(precision=6, suppress=True)

In [352]:
reg.coef_.argmin()

10

In [353]:
reg.coef_.argmax()

5

In [354]:
reg.coef_

array([ 124534.875 , -231908.47  , -186481.39  ,   69238.336 ,
        236928.3   , 1554221.1   ,    6384.124 ,    5420.8066,
        -86145.5   ,   -2723.7769, -389396.56  , -270560.    ,
        371938.88  , -363135.28  , -214997.8   , -365170.22  ,
        -15475.641 ,  375300.38  , -355484.72  , -262487.22  ],
      dtype=float32)

In [419]:
reg.coef_.shape

(20,)

In [365]:
top_topics[0][0][0]

(0.008376592, 'lens')

In [371]:
tps = []
for topic in top_topics:
    tmp = []
    ts = topic[0]
    for t in ts:
        tmp.append(t[1])
        if len(tmp) == 10:
            break
    tmp = ', '.join(tmp)
    tps.append(tmp)

In [379]:
ldf = pd.DataFrame(model.print_topics())
ldf

,0,1
0,0,"0.013*""iphone"" + 0.006*""im"" + 0.005*""device"" +..."
1,1,"0.018*""pro"" + 0.013*""iphone"" + 0.011*""max"" + 0..."
2,2,"0.006*""photo"" + 0.006*""would"" + 0.006*""fe"" + 0..."
3,3,"0.007*""going"" + 0.006*""display"" + 0.005*""would..."
4,4,"0.008*""case"" + 0.006*""photo"" + 0.005*""x"" + 0.0..."
5,5,"0.006*""series"" + 0.005*""performance"" + 0.005*""..."
6,6,"0.006*""music"" + 0.006*""still"" + 0.005*""perform..."
7,7,"0.012*""right"" + 0.006*""device"" + 0.006*""going""..."
8,8,"0.009*""iphone"" + 0.008*""pro"" + 0.007*""photo"" +..."
9,9,"0.012*""iphone"" + 0.009*""pro"" + 0.007*""megapixe..."


In [382]:
ldf.columns = ['Topic', 'Representation']
ldf

,Topic,Representation
0,0,"0.013*""iphone"" + 0.006*""im"" + 0.005*""device"" +..."
1,1,"0.018*""pro"" + 0.013*""iphone"" + 0.011*""max"" + 0..."
2,2,"0.006*""photo"" + 0.006*""would"" + 0.006*""fe"" + 0..."
3,3,"0.007*""going"" + 0.006*""display"" + 0.005*""would..."
4,4,"0.008*""case"" + 0.006*""photo"" + 0.005*""x"" + 0.0..."
5,5,"0.006*""series"" + 0.005*""performance"" + 0.005*""..."
6,6,"0.006*""music"" + 0.006*""still"" + 0.005*""perform..."
7,7,"0.012*""right"" + 0.006*""device"" + 0.006*""going""..."
8,8,"0.009*""iphone"" + 0.008*""pro"" + 0.007*""photo"" +..."
9,9,"0.012*""iphone"" + 0.009*""pro"" + 0.007*""megapixe..."


In [414]:
cnt = []
for i in range(20):
    cnt.append(topics.count(i))

In [415]:
topics.count(-1)

0

In [390]:
ldf.insert(loc=1, column='Count', value=cnt)

In [416]:
ldf['Count'] = cnt
ldf

,Topic,Count,Representation
0,0,73,"0.013*""iphone"" + 0.006*""im"" + 0.005*""device"" +..."
1,1,32,"0.018*""pro"" + 0.013*""iphone"" + 0.011*""max"" + 0..."
2,2,26,"0.006*""photo"" + 0.006*""would"" + 0.006*""fe"" + 0..."
3,3,40,"0.007*""going"" + 0.006*""display"" + 0.005*""would..."
4,4,26,"0.008*""case"" + 0.006*""photo"" + 0.005*""x"" + 0.0..."
5,5,53,"0.006*""series"" + 0.005*""performance"" + 0.005*""..."
6,6,161,"0.006*""music"" + 0.006*""still"" + 0.005*""perform..."
7,7,22,"0.012*""right"" + 0.006*""device"" + 0.006*""going""..."
8,8,54,"0.009*""iphone"" + 0.008*""pro"" + 0.007*""photo"" +..."
9,9,29,"0.012*""iphone"" + 0.009*""pro"" + 0.007*""megapixe..."


In [420]:
ldf['Coef'] = reg.coef_
ldf

,Topic,Count,Representation,Coef
0,0,73,"0.013*""iphone"" + 0.006*""im"" + 0.005*""device"" +...",124534.8750
1,1,32,"0.018*""pro"" + 0.013*""iphone"" + 0.011*""max"" + 0...",-231908.4688
2,2,26,"0.006*""photo"" + 0.006*""would"" + 0.006*""fe"" + 0...",-186481.3906
3,3,40,"0.007*""going"" + 0.006*""display"" + 0.005*""would...",69238.3359
4,4,26,"0.008*""case"" + 0.006*""photo"" + 0.005*""x"" + 0.0...",236928.2969
5,5,53,"0.006*""series"" + 0.005*""performance"" + 0.005*""...",1554221.1250
6,6,161,"0.006*""music"" + 0.006*""still"" + 0.005*""perform...",6384.1240
7,7,22,"0.012*""right"" + 0.006*""device"" + 0.006*""going""...",5420.8066
8,8,54,"0.009*""iphone"" + 0.008*""pro"" + 0.007*""photo"" +...",-86145.5000
9,9,29,"0.012*""iphone"" + 0.009*""pro"" + 0.007*""megapixe...",-2723.7769


In [422]:
ldf2 = ldf.sort_values(by='Coef', ascending=False)

In [423]:
ldf2

,Topic,Count,Representation,Coef
5,5,53,"0.006*""series"" + 0.005*""performance"" + 0.005*""...",1554221.1250
17,17,8,"0.010*""little"" + 0.007*""music"" + 0.007*""watch""...",375300.3750
12,12,28,"0.013*""iphone"" + 0.007*""pro"" + 0.007*""city"" + ...",371938.8750
4,4,26,"0.008*""case"" + 0.006*""photo"" + 0.005*""x"" + 0.0...",236928.2969
0,0,73,"0.013*""iphone"" + 0.006*""im"" + 0.005*""device"" +...",124534.8750
3,3,40,"0.007*""going"" + 0.006*""display"" + 0.005*""would...",69238.3359
6,6,161,"0.006*""music"" + 0.006*""still"" + 0.005*""perform...",6384.1240
7,7,22,"0.012*""right"" + 0.006*""device"" + 0.006*""going""...",5420.8066
9,9,29,"0.012*""iphone"" + 0.009*""pro"" + 0.007*""megapixe...",-2723.7769
16,16,35,"0.008*""photo"" + 0.007*""little"" + 0.007*""iphone...",-15475.6406


In [424]:
ldf2.to_csv('galaxy_show_lda.csv',index=False)

In [ ]:
tps = []
for topic in top_topics:
    tmp = []
    ts = topic[1]

In [401]:
[list(dict(model.get_document_topics(c, minimum_probability=0)).values()) for c in corpus]

[[0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.704643,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.00018200994,
  0.2920808,
  0.00018200994,
  0.00018200994,
  0.00018200994],
 [4.7200232e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  0.99910325,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05,
  4.720023e-05],
 [2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  0.83617175,
  2.371877e-05,
  0.16340137,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05,
  2.371877e-05],
 [0.00043120005,
  0.0004312000

In [413]:
topics = pd.DataFrame(prob_arr).idxmax(axis=1).tolist()

In [398]:
document_topic_matrix = [list(dict(model.get_document_topics(doc, minimum_probability=0)).values()) for doc in corpus]

array([[0.000342, 0.000081, 0.000637, ..., 0.000008, 0.000012, 0.      ],
       [0.000101, 0.000004, 0.0003  , ..., 0.000004, 0.000004, 0.000004],
       [0.000339, 0.00037 , 0.000585, ..., 0.000003, 0.000003, 0.000003],
       ...,
       [0.000419, 0.000008, 0.000008, ..., 0.000008, 0.000008, 0.000008],
       [0.000263, 0.000071, 0.000496, ..., 0.000004, 0.000004, 0.000004],
       [0.000261, 0.000088, 0.000113, ..., 0.000002, 0.000002, 0.000002]],
      dtype=float32)

In [372]:
pd.DataFrame(columns=['Topic']

0     lens, think, iphone, little, pro, bit, im, goi...
1     iphone, im, device, little, photo, zoom, color...
2     iphone, pro, city, going, case, little, right,...
3     music, still, performance, iphone, series, dis...
4     iphone, pro, still, max, device, apple, think,...
5     iphone, case, smartphone, pro, gb, course, pho...
6     case, megapixel, design, x, display, zoom, cha...
7     going, display, would, mode, x, mobile, series...
8     series, performance, think, still, iphone, mod...
9     iphone, pro, megapixel, zoom, shot, think, lit...
10    iphone, pro, photo, light, zoom, mode, little,...
11    pro, iphone, pixel, im, right, price, think, e...
12    right, device, going, im, pro, little, photo, ...
13    photo, little, opa, everything, main, right, w...
14    case, photo, x, going, zoom, pen, best, smartp...
15    pro, iphone, max, series, apple, india, differ...
16    photo, little, iphone, right, difference, need...
17    photo, would, fe, little, expert, still, m

In [214]:
joinp = [' '.join(doc) for doc in processed_data]

In [215]:
joinp[0]

'ultra official introduction film whats new new ultra share epic powerful processor yet prograde camera boast epic hi nothing bad right whats matter braided ultra good expressed simply good first display color contrast automatically adjusted match content watching surrounding environment function called vision booster oh look drive look snapdragon nd generation let increase gpu cpu npu powerful processor stop stop stop cant stop processor efficient keep going moment water fountain concentrate ultra product take earth consideration consists least recycled ocean plastic average recycled glass least recycled pet film cool perfect space astrophotography expert raw expert wait sec oh cool pen pen powerful pen finetune color curve exposure etc let take selfie huh nytography ai object recognition engine recognizes detail skin eyebrow hair send picture thank sending wheres sled need fun photograph nightography ultra feature twicelarger ois improved ai stable night photography film look incredi

In [236]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, max_cluster_size=100, prediction_data=True)
topic_model = BERTopic(hdbscan_model=hdbscan_model, calculate_probabilities=True, low_memory=True, min_topic_size=5)

In [ ]:
topics, probs = topic_model.fit_transform(joinp)
topic_model.get_topic_info()

In [238]:
topic_model = BERTopic(calculate_probabilities=True, low_memory=True, min_topic_size=5)

In [239]:
topics, probs = topic_model.fit_transform(joinp)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,301,-1_ultra_camera_like_phone,"[ultra, camera, like, phone, also, one, video,...",[ultra v iphone pro day win product link casek...
1,0,58,0_tap_screen_button_want,"[tap, screen, button, want, screenshot, go, fu...",[top trucos tip te compraste un nuevo quieres ...
2,1,44,1_fe_sfe_edition_fan,"[fe, sfe, edition, fan, exynos, launch, series...",[fe yesthey finally fe waynow dropped fe compa...
3,2,38,2_snapdragon_exynos_chip_qualcomm,"[snapdragon, exynos, chip, qualcomm, cpu, chip...",[ultra confirmed ultra confirmed bring back ex...
4,3,35,3_ultra_series_plus_new,"[ultra, series, plus, new, leaked, storage, ca...",[series exclusive leak everything far series e...
5,4,34,4_like_unboxing_phone_one,"[like, unboxing, phone, one, camera, good, box...",[unboxing amp first impressionsthe perfect com...
6,5,30,5_camera_photo_lens_photograph,"[camera, photo, lens, photograph, expert, see,...",[ultra photographer review go httpcasetifycomt...
7,6,29,6_phone_good_also_performance,"[phone, good, also, performance, display, get,...",[unabhngig amp gut amp plus im test tiefgreife...
8,7,28,7_device_gb_also_ultra,"[device, gb, also, ultra, screen, camera, supp...",[e ultra handson nova famlia chegou relevou o ...
9,8,22,8_case_protection_protector_designed,"[case, protection, protector, designed, cases,...",[top best ultra case spigen caseology ringke w...


In [427]:
fig = topic_model.visualize_topics()

In [428]:
fig.write_html("bertopic.html")

In [240]:
new_topics = topic_model.reduce_outliers(joinp, topics)

In [272]:
df = pd.DataFrame(topic_model.get_topic_info())

In [286]:
df = df.drop(0)

In [291]:
df['Coef'] = reg.coef_

In [293]:
pd.set_option('display.float_format', lambda x: f'{x:.4f}') 

In [301]:
df['Topic'] = df['Topic'] - 1

In [302]:
df

,Topic,Count,Name,Representation,Representative_Docs,Coef
1,0,58,0_tap_screen_button_want,"[tap, screen, button, want, screenshot, go, fu...",[top trucos tip te compraste un nuevo quieres ...,-463284.1499
2,1,44,1_fe_sfe_edition_fan,"[fe, sfe, edition, fan, exynos, launch, series...",[fe yesthey finally fe waynow dropped fe compa...,-569617.0732
3,2,38,2_snapdragon_exynos_chip_qualcomm,"[snapdragon, exynos, chip, qualcomm, cpu, chip...",[ultra confirmed ultra confirmed bring back ex...,-481736.0791
4,3,35,3_ultra_series_plus_new,"[ultra, series, plus, new, leaked, storage, ca...",[series exclusive leak everything far series e...,-529034.4251
5,4,34,4_like_unboxing_phone_one,"[like, unboxing, phone, one, camera, good, box...",[unboxing amp first impressionsthe perfect com...,225173.4199
6,5,30,5_camera_photo_lens_photograph,"[camera, photo, lens, photograph, expert, see,...",[ultra photographer review go httpcasetifycomt...,-339660.4205
7,6,29,6_phone_good_also_performance,"[phone, good, also, performance, display, get,...",[unabhngig amp gut amp plus im test tiefgreife...,-135453.4561
8,7,28,7_device_gb_also_ultra,"[device, gb, also, ultra, screen, camera, supp...",[e ultra handson nova famlia chegou relevou o ...,-597045.5681
9,8,22,8_case_protection_protector_designed,"[case, protection, protector, designed, cases,...",[top best ultra case spigen caseology ringke w...,-490976.2797
10,9,16,9_iphone_max_pro_apple,"[iphone, max, pro, apple, titanium, height, ul...",[iphone pro max v ultra le gros comparatif leq...,396298.7261


In [303]:
df.to_csv("galaxy.csv", index=False)

In [304]:
(df.sort_values(by='Coef', ascending=False)).to_csv("galaxy_coef.csv", index=False, encoding='utf-8')

In [330]:
df2 = df.sort_values(by='Coef', ascending=False)
df2

,Topic,Count,Name,Representation,Representative_Docs,Coef
32,31,7,31_moon_photo_fake_detail,"[moon, photo, fake, detail, image, ai, zoom, r...",[ultra faking ultra best camera setup smartpho...,1276756.3221
22,21,11,21_ultra_glass_series_experience,"[ultra, glass, series, experience, using, book...",[ultra durability test close eye today time du...,1050217.7730
27,26,8,26_phone_going_like_phones,"[phone, going, like, phones, get, thats, prett...",[flagship v midrange debate review close midra...,549133.7145
10,9,16,9_iphone_max_pro_apple,"[iphone, max, pro, apple, titanium, height, ul...",[iphone pro max v ultra le gros comparatif leq...,396298.7261
12,11,15,11_yeah_cellphone_like_yes,"[yeah, cellphone, like, yes, thats, android, o...",[sultra g official price g official price boug...,337719.6560
5,4,34,4_like_unboxing_phone_one,"[like, unboxing, phone, one, camera, good, box...",[unboxing amp first impressionsthe perfect com...,225173.4199
14,13,15,13_charging_test_pro_battery,"[charging, test, pro, battery, iphone, charger...",[iphone pro max v ultra v iphone pro max batte...,212552.4898
13,12,15,12_iphone_max_pro_portrait,"[iphone, max, pro, portrait, zoom, ultra, vivo...",[way iphone pro max v ultra camera battery tes...,667.8060
17,16,13,16_iphone_performance_ultra_effect,"[iphone, performance, ultra, effect, however, ...",[ultra v iphone pro sultra hk ultra time take ...,-33110.3117
7,6,29,6_phone_good_also_performance,"[phone, good, also, performance, display, get,...",[unabhngig amp gut amp plus im test tiefgreife...,-135453.4561


In [331]:
df['Representation'].iloc[0][:5]

['tap', 'screen', 'button', 'want', 'screenshot']

In [332]:
def top5(words):
    return ', '.join(words[:10])

In [333]:
df['Representation'].apply(top5)

1     tap, screen, button, want, screenshot, go, fun...
2     fe, sfe, edition, fan, exynos, launch, series,...
3     snapdragon, exynos, chip, qualcomm, cpu, chips...
4     ultra, series, plus, new, leaked, storage, cam...
5     like, unboxing, phone, one, camera, good, box,...
6     camera, photo, lens, photograph, expert, see, ...
7     phone, good, also, performance, display, get, ...
8     device, gb, also, ultra, screen, camera, suppo...
9     case, protection, protector, designed, cases, ...
10    iphone, max, pro, apple, titanium, height, ult...
11    su, phone, need, also, go, turn, video, see, s...
12    yeah, cellphone, like, yes, thats, android, on...
13    iphone, max, pro, portrait, zoom, ultra, vivo,...
14    charging, test, pro, battery, iphone, charger,...
15    ultra, camera, new, megapixels, smartphone, fe...
16    battery, life, ultra, power, hour, time, use, ...
17    iphone, performance, ultra, effect, however, a...
18    xiaomi, bit, second, per, iphone, recordin

In [334]:
df2['Representation'] = df2['Representation'].apply(top5)

In [335]:
df2 = df2[['Topic', 'Count', 'Name', 'Representation', 'Coef']]
df2

,Topic,Count,Name,Representation,Coef
32,31,7,31_moon_photo_fake_detail,"moon, photo, fake, detail, image, ai, zoom, re...",1276756.3221
22,21,11,21_ultra_glass_series_experience,"ultra, glass, series, experience, using, book,...",1050217.7730
27,26,8,26_phone_going_like_phones,"phone, going, like, phones, get, thats, pretty...",549133.7145
10,9,16,9_iphone_max_pro_apple,"iphone, max, pro, apple, titanium, height, ult...",396298.7261
12,11,15,11_yeah_cellphone_like_yes,"yeah, cellphone, like, yes, thats, android, on...",337719.6560
5,4,34,4_like_unboxing_phone_one,"like, unboxing, phone, one, camera, good, box,...",225173.4199
14,13,15,13_charging_test_pro_battery,"charging, test, pro, battery, iphone, charger,...",212552.4898
13,12,15,12_iphone_max_pro_portrait,"iphone, max, pro, portrait, zoom, ultra, vivo,...",667.8060
17,16,13,16_iphone_performance_ultra_effect,"iphone, performance, ultra, effect, however, a...",-33110.3117
7,6,29,6_phone_good_also_performance,"phone, good, also, performance, display, get, ...",-135453.4561


In [336]:
df2.to_csv('galaxy_show.csv', index=False)

In [276]:
reg.coef_.shape

(36,)

In [244]:
probs.shape

(901, 36)

In [279]:
reg.coef_

array([-463284.149905, -569617.073249, -481736.079056, -529034.425099,
        225173.419901, -339660.420542, -135453.456123, -597045.568062,
       -490976.279691,  396298.726128, -423666.399406,  337719.655966,
           667.805964,  212552.489756, -616571.888286, -531995.181135,
        -33110.311723, -357643.52272 , -510005.953382, -240273.283792,
       -188952.055233, 1050217.772975, -410225.89646 , -552891.922034,
       -477650.313444, -429639.530271,  549133.714469, -481615.669704,
       -575409.777381, -180265.98913 , -483742.176676, 1276756.322065,
       -319567.092955, -574363.397091, -533861.060588, -506270.740103])